In [104]:
class Sentiment:
    NEGATIVE= 'NEGATIVE'
    NEUTRAL = 'NEUTRAL'
    POSITIVE = 'POSITIVE'



class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <=2:
            return Sentiment.NEGATIVE
        if self.score ==3:
            return Sentiment.NEUTRAL
        if self.score >3:
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__ (self, reviews):
        self.reviews = reviews
        
    def evenly_distribute(self):
        negative = filter(lambda x:x.sentiment == Sentimemnt.NEGATIVE, self.reviews)
        positive = filter(lambda x:x.sentiment == Sentimemnt.POSITIVE, self.reviews)
        
    

In [95]:
import pandas as pd
import json
import numpy as np

file_name = r'C:\Users\asus\Desktop\stats\books_small_10000.json'
reviews=[]
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))


       

In [96]:
reviews[25].text

"Wow...just wow!!!! Omg I absolutely love this sensitive side to Grant!!! My heart hurt for both him and Harlow throughout the book!! I seriously and I mean SERIOUSLY still can't stand Nan!!! I need more....god the wait for book two is gonna kill me!!! You kick some serious a** Abbi and you are one of my absolute favorite authors!!! Thank you so much for writing this series!!!"

In [47]:
#bag of words method for text in models - Read

# Splitting the data

In [98]:
from sklearn.model_selection import train_test_split
training, test = train_test_split(reviews, test_size = 0.33, random_state = 42)
cont = ReviewContainer(training)

In [99]:
len(training)

6700

In [100]:
len(test)

3300

In [101]:
train_x= [x.text for x in training]
train_y= [x.sentiment for x in training]

test_x= [x.text for x in test]
test_y= [x.sentiment for x in test]

print(train_x[0])
print(train_y[0])

Olivia Hampton arrives at the Dunraven family home as cataloger of their extensive library. What she doesn't expect is a broken carriage wheel on the way. Nor a young girl whose mind is clearly gone, an old man in need of care himself (and doesn&#8217;t quite seem all there in Olivia&#8217;s opinion). Furthermore, Marion Dunraven, the only sane one of the bunch and the one Olivia is inexplicable drawn to, seems captive to everyone in the dusty old house. More importantly, she doesn't expect to fall in love with Dunraven's daughter Marion.Can Olivia truly believe the stories of sadness and death that surround the house, or are they all just local neighborhood rumor?Was that carriage trouble just a coincidence or a supernatural sign to stay away? If she remains, will the Castle&#8217;s dark shadows take Olivia down with them or will she and Marion long enough to declare their love?Patty G. Henderson has created an atmospheric and intriguing story in her Gothic tale. I found this to be an

# Bag of words

In [102]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
#fit_transform is the same as first fit and then transform
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)
train_x_vectors[0]



<1x26615 sparse matrix of type '<class 'numpy.int64'>'
	with 131 stored elements in Compressed Sparse Row format>

# Classification

### Linear SVM

In [66]:
from sklearn.svm import SVC

clf_svm = SVC(kernel = 'linear')

clf_svm.fit(train_x_vectors ,train_y)

print(test_x[0])

clf_svm.predict(test_x_vectors[0])



Every new Myke Cole book is better than the last, and this is no exception. If you haven't read the Shadow Ops series before start with Control Point, but go ahead and order Fortress Frontier and Breach Zone as well - you're going to want them.


array(['POSITIVE'], dtype='<U8')

### Decision tree

In [68]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()

clf_dec.fit(train_x_vectors ,train_y)

clf_dec.predict(test_x_vectors[0])


array(['POSITIVE'], dtype='<U8')

### Naive Bayes

In [76]:
from sklearn.naive_bayes import GaussianNB

clf_GNB =  GaussianNB()
train_x_array = train_x_vectors.toarray()
test_x_array = train_x_vectors.toarray()
clf_GNB.fit(train_x_array ,train_y)


array(['POSITIVE'], dtype='<U8')

In [77]:
### Logistic regression

In [78]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors ,train_y)

clf_dec.predict(test_x_vectors[0])

C:\Users\asus\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array(['POSITIVE'], dtype='<U8')

# Evaluation

In [79]:
clf_svm.score(test_x_vectors, test_y)

0.8242424242424242

In [84]:
clf_dec.score(test_x_vectors, test_y)

0.7575757575757576

In [87]:
clf_log.score(test_x_vectors, test_y)

0.8303030303030303

Not testing for GNB because of multiple conversions.

In [88]:
# f1 scores

from sklearn.metrics import f1_score
f1_score(test_y, clf_svm.predict(test_x_vectors), average = None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])

array([0.91319444, 0.21052632, 0.22222222])

In [90]:

f1_score(test_y, clf_log.predict(test_x_vectors), average = None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])

array([0.91370558, 0.12244898, 0.1       ])

# Improving the model

In [103]:
train_y.count(Sentiment.NEGATIVE)

436